In [ ]:
import sys
sys.path.append('/Users/czkaiweb/Research/ErdosBootCamp/May2022/vanGogh-and-Other-Artist')
from genericCNN import *
from preprocessing.ImageTranform import *
from torchsummary import summary

import os

In [ ]:
myObj = genericCNN()

# Set up the transformer
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()

myObj.setTransformer(transformer)
# Decide if adding normalization layer at the end of transformation, by default, normalization will be added
# myObj.UseNormalized(normalize = True)

In [ ]:
# Set up the meta data and path to image dataset
myObj.setDataset("../data/meta.csv",path = "../data/imgs")

# Split the data by portion, fraction indicate the percentage of data used in the whole dataset. 
# Default: val_size = 0.2, test_size = 0.1 
myObj.splitData(val_size=0.02,test_size = 0.9,fraction = 0.1)

# Will automatically get the statistic for training set, update the mean/std used for normalization. 
# loadData and checkDataset
myObj.loadData(reUseTrain = 3)

In [ ]:
myObj.showDatasetBatch()

In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

# Try to load model with pretrained first!
# model_ft = models.vgg16(pretrained= True)

#If the pretrained argument doesn't work, download the weight file to local ./weights and load from local file
model_ft = models.vgg16()
model_ft.load_state_dict(torch.load('./weights/vgg16-397923af.pth'))

num_ftrs = model_ft.classifier[6].in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.classifier[6] = nn.Linear(num_ftrs, 6)

model_ft = model_ft.to(myObj.device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
myObj.setModel(model = model_ft,modeltag="vgg16mod")

In [ ]:
myObj.train_model(criterion, optimizer_ft, exp_lr_scheduler, num_epochs=2)

In [ ]:
myObj.evaluate(saveAs="output.png")

In [ ]:
myObj.drawHistory()

In [ ]:
weight_path = "./postTrain/"

if not os.path.isdir(weight_path):
    os.mkdir(weight_path)

torch.save(myObj.Model.state_dict(), weight_path + 'VGG_postTrain_weights_May25.pth')